# Week 4: A Detour into SQL with SQLite

This week we step away from Spark to explore **relational databases** using **SQLite** — a lightweight, file-based SQL database that ships built into Python.

We will use real data from **[POGOH](https://pogoh.com/)**, Pittsburgh's bike-share system, which contains over **one million trips** taken between 2024 and 2025.

### What we'll cover
1. Database design & normalization
2. Creating tables and indices in SQLite
3. Bulk-loading data from a CSV
4. SQL queries: `SELECT`, `WHERE`, `JOIN`, `GROUP BY`, `HAVING`
5. Time-based queries with `strftime()`
6. How indices make lookups faster

## Setup

In [31]:
import sqlite3
import os
import time
import pandas as pd

CSV_PATH = '../data/pogoh_full.csv'
DB_PATH  = '../data/pogoh.db'

# If running on Google Colab, mount your Drive and adjust the paths above:
# from google.colab import drive
# drive.mount('/content/drive')
# CSV_PATH = '/content/drive/MyDrive/...'

## 1  The POGOH Dataset

Let's load the raw CSV with pandas and take a first look.

In [32]:
df = pd.read_csv(CSV_PATH)
print(f"Shape: {df.shape[0]:,} rows  x  {df.shape[1]} columns")
df.head()

Shape: 1,034,643 rows  x  10 columns


,Closed Status,Duration,Start Station Id,Start Date,Start Station Name,End Date,End Station Id,End Station Name,Rider Type,source_file
0,NORMAL,299,51,2024-04-30 23:53:00,Coltart Ave & Forbes Ave,2024-04-30 23:58:00,20.0,Boulevard of the Allies & Parkview Ave,MEMBER,april-2024.xlsx
1,NORMAL,399,28,2024-04-30 23:49:00,Fifth Ave & S Bouquet St,2024-04-30 23:56:00,10.0,Zulema St & Coltart Ave,MEMBER,april-2024.xlsx
2,NORMAL,409,34,2024-04-30 23:46:00,N Dithridge St & Centre Ave,2024-04-30 23:53:00,20.0,Boulevard of the Allies & Parkview Ave,MEMBER,april-2024.xlsx
3,NORMAL,348,10,2024-04-30 23:43:00,Zulema St & Coltart Ave,2024-04-30 23:48:00,10.0,Zulema St & Coltart Ave,MEMBER,april-2024.xlsx
4,NORMAL,248,13,2024-04-30 23:42:00,S Bouquet Ave & Sennott St,2024-04-30 23:46:00,20.0,Boulevard of the Allies & Parkview Ave,MEMBER,april-2024.xlsx


In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1034643 entries, 0 to 1034642
Data columns (total 10 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   Closed Status       1034643 non-null  object 
 1   Duration            1034643 non-null  int64  
 2   Start Station Id    1034643 non-null  int64  
 3   Start Date          1034643 non-null  object 
 4   Start Station Name  1034643 non-null  object 
 5   End Date            1034643 non-null  object 
 6   End Station Id      1034331 non-null  float64
 7   End Station Name    1034331 non-null  object 
 8   Rider Type          1034643 non-null  object 
 9   source_file         1034643 non-null  object 
dtypes: float64(1), int64(2), object(7)
memory usage: 78.9+ MB


In [34]:
# Explore the low-cardinality categorical columns
for col in ['Closed Status', 'Rider Type']:
    counts = df[col].value_counts()
    print(f"\n--- {col} ---")
    print(counts.to_string())


--- Closed Status ---
Closed Status
NORMAL           988470
GRACE_PERIOD      44324
FORCED_CLOSED      1119
TERMINATED          730

--- Rider Type ---
Rider Type
MEMBER    954450
CASUAL     80193


In [35]:
# Trip duration is stored in seconds
print(df['Duration'].describe())
print(f"\nMax duration: {df['Duration'].max() / 3600:.1f} hours")

count    1.034643e+06
mean     8.198733e+02
std      3.593489e+03
min      0.000000e+00
25%      2.290000e+02
50%      3.950000e+02
75%      8.260000e+02
max      1.290684e+06
Name: Duration, dtype: float64

Max duration: 358.5 hours


In [36]:
# How many rows have a missing end station?
missing = df['End Station Id'].isna().sum()
print(f"Rows missing End Station Id: {missing:,}  ({missing/len(df)*100:.2f}%)")

# Drop them -- they are < 0.03% of trips, negligible
df = df.dropna(subset=['End Station Id', 'End Station Name']).copy()
df['Start Station Id'] = df['Start Station Id'].astype(int)
df['End Station Id']   = df['End Station Id'].astype(int)
print(f"Rows after cleaning: {len(df):,}")

Rows missing End Station Id: 312  (0.03%)
Rows after cleaning: 1,034,331


## 2  Database Design

### Why normalize?

The raw CSV stores `Start Station Name` and `End Station Name` on **every single row**.  
That is redundant: if a station is renamed, we would have to update a million rows.

Instead, we split the data into two tables connected by **foreign keys**:

```
+-------------------------+         +----------------------------------------+
|        stations         |         |                 trips                  |
+-------------------------+         +----------------------------------------+
| station_id   INTEGER PK |<---+    | trip_id          INTEGER PK            |
| station_name TEXT       |   +---->| start_station_id INTEGER FK -> stations|
+-------------------------+   +---->| end_station_id   INTEGER FK -> stations|
                              |    | closed_status    TEXT                  |
                              +----| duration_sec     INTEGER               |
                                   | start_time       TEXT  (ISO datetime)  |
                                   | end_time         TEXT  (ISO datetime)  |
                                   | rider_type       TEXT                  |
                                   +----------------------------------------+
```

### Why create indices?

An index is like the index at the back of a textbook: instead of scanning every page (row) to find a topic, you jump directly to the right location.  
We add indices on columns that we frequently **filter** (`WHERE`), **sort** (`ORDER BY`), or **join** on.

| Index | Column | Useful for |
|---|---|---|
| `idx_trips_start` | `start_station_id` | JOINs, "trips departing from station X" |
| `idx_trips_end` | `end_station_id` | JOINs, "trips arriving at station X" |
| `idx_trips_rider` | `rider_type` | Filtering MEMBER vs CASUAL |
| `idx_trips_time` | `start_time` | Date-range queries |

> **Trade-off**: indices make reads faster but slow down inserts slightly and use extra disk space.  
> That is why we create them *after* loading the data.

## 3  Creating the Database

In [38]:
# Remove any old copy so we start fresh
if os.path.exists(DB_PATH):
    os.remove(DB_PATH)

conn = sqlite3.connect(DB_PATH)
cur  = conn.cursor()
print("Connected to:", DB_PATH)

Connected to: ../data/pogoh.db


In [39]:
# Create tables (DDL = Data Definition Language)
cur.executescript("""
CREATE TABLE stations (
    station_id   INTEGER PRIMARY KEY,
    station_name TEXT    NOT NULL
);

CREATE TABLE trips (
    trip_id          INTEGER PRIMARY KEY AUTOINCREMENT,
    closed_status    TEXT    NOT NULL,
    duration_sec     INTEGER NOT NULL,
    start_station_id INTEGER NOT NULL REFERENCES stations(station_id),
    start_time       TEXT    NOT NULL,
    end_station_id   INTEGER NOT NULL REFERENCES stations(station_id),
    end_time         TEXT    NOT NULL,
    rider_type       TEXT    NOT NULL
);
""")

conn.commit()
print("Tables created.")

Tables created.


## 4  Loading Data

We load stations first (only 62 unique ones), then the full trip table.  
Notice that `trips` stores only **integer IDs** for stations -- that is the whole point of normalization.

In [40]:
# Build the stations table from both the start- and end-station columns
start_s = df[['Start Station Id', 'Start Station Name']].rename(
    columns={'Start Station Id': 'station_id', 'Start Station Name': 'station_name'})
end_s   = df[['End Station Id',   'End Station Name']].rename(
    columns={'End Station Id':   'station_id', 'End Station Name':   'station_name'})

stations_df = (
    pd.concat([start_s, end_s])
      .drop_duplicates(subset='station_id')
      .sort_values('station_id')
      .reset_index(drop=True)
)

print(f"{len(stations_df)} unique stations")
stations_df.head()

61 unique stations


,station_id,station_name
0,1,Pierce St & Summerlea St
1,2,Eliza Furnace Trail & Swineburne St
2,3,Centre Ave & Addison St
3,4,Burns White Center at 3 Crossings
4,5,Allegheny Station


In [41]:
cur.executemany(
    "INSERT INTO stations (station_id, station_name) VALUES (?, ?)",
    stations_df.itertuples(index=False, name=None)
)
conn.commit()
print("Stations loaded.")

Stations loaded.


In [42]:
# Rename columns to match the trips schema and drop source_file (not needed in the DB)
trips_df = df[['Closed Status', 'Duration',
               'Start Station Id', 'Start Date',
               'End Station Id',   'End Date',
               'Rider Type']].copy()

trips_df.columns = ['closed_status', 'duration_sec',
                    'start_station_id', 'start_time',
                    'end_station_id',   'end_time',
                    'rider_type']

print(f"Inserting {len(trips_df):,} trips ... (this may take ~30 seconds)")
t0 = time.time()

# pandas to_sql is the fastest way to bulk-insert into SQLite
trips_df.to_sql('trips', conn, if_exists='append', index=False)
conn.commit()

print(f"Done in {time.time()-t0:.1f}s")

Inserting 1,034,331 trips ... (this may take ~30 seconds)
Done in 1.2s


In [43]:
# Build indices AFTER bulk insert -- it is faster to index a full table once
# than to update the index on every inserted row

print("Creating indices ...")
t0 = time.time()

cur.executescript("""
CREATE INDEX idx_trips_start ON trips(start_station_id);
CREATE INDEX idx_trips_end   ON trips(end_station_id);
CREATE INDEX idx_trips_rider ON trips(rider_type);
CREATE INDEX idx_trips_time  ON trips(start_time);
""")
conn.commit()

print(f"Indices built in {time.time()-t0:.1f}s")

Creating indices ...
Indices built in 1.2s


In [44]:
# Sanity check
for table in ('stations', 'trips'):
    n = cur.execute(f"SELECT COUNT(*) FROM {table}").fetchone()[0]
    print(f"{table}: {n:,} rows")

stations: 61 rows
trips: 1,034,331 rows


## 5  Basic SQL Queries

We use `pd.read_sql()` to run queries and display results as DataFrames.  
Under the hood this is exactly the same SQL any other client would send.

In [45]:
# SELECT * -- retrieve every row and column
pd.read_sql("SELECT * FROM stations ORDER BY station_id", conn)

,station_id,station_name
0,1,Pierce St & Summerlea St
1,2,Eliza Furnace Trail & Swineburne St
2,3,Centre Ave & Addison St
3,4,Burns White Center at 3 Crossings
4,5,Allegheny Station
...,...,...
56,57,52nd St & Butler St
57,58,W North Ave & Federal St
58,59,Forbes Ave at TCS Hall (CMU Campus)
59,60,Wilkinsburg Park & Ride


In [46]:
# WHERE -- filter rows
pd.read_sql("""
    SELECT *
    FROM   trips
    WHERE  rider_type = 'CASUAL'
    LIMIT  10
""", conn)

,trip_id,closed_status,duration_sec,start_station_id,start_time,end_station_id,end_time,rider_type
0,9,NORMAL,1051,6,2024-04-30 23:22:00,4,2024-04-30 23:39:00,CASUAL
1,10,NORMAL,1102,6,2024-04-30 23:21:00,4,2024-04-30 23:39:00,CASUAL
2,13,NORMAL,625,40,2024-04-30 23:17:00,23,2024-04-30 23:28:00,CASUAL
3,17,NORMAL,6783,6,2024-04-30 23:11:00,54,2024-05-01 01:04:00,CASUAL
4,22,NORMAL,532,55,2024-04-30 22:52:00,40,2024-04-30 23:00:00,CASUAL
5,23,GRACE_PERIOD,25,55,2024-04-30 22:51:00,55,2024-04-30 22:51:00,CASUAL
6,25,GRACE_PERIOD,26,55,2024-04-30 22:50:00,55,2024-04-30 22:50:00,CASUAL
7,29,NORMAL,4292,26,2024-04-30 22:46:00,26,2024-04-30 23:57:00,CASUAL
8,30,NORMAL,4309,26,2024-04-30 22:45:00,26,2024-04-30 23:57:00,CASUAL
9,33,NORMAL,2064,23,2024-04-30 22:41:00,40,2024-04-30 23:16:00,CASUAL


In [ ]:
# ORDER BY + LIMIT -- find the longest trips
import time
start_time = time.time()
df = pd.read_sql("""
    SELECT trip_id,
           start_station_id,
           end_station_id,
           ROUND(duration_sec / 3600.0, 2) AS duration_hours,
           rider_type
    FROM   trips
    ORDER  BY duration_hours DESC
    LIMIT  100
""", conn)
print(f"Query executed in {time.time() - start_time:.5f} seconds")
df

Query executed in 1.68951 seconds


,trip_id,start_station_id,end_station_id,duration_hours,rider_type
0,845850,51,24,358.52,CASUAL
1,154039,32,24,192.97,MEMBER
2,875915,21,57,167.63,MEMBER
3,986838,59,52,141.72,MEMBER
4,164884,21,24,134.15,MEMBER
...,...,...,...,...,...
1034326,1033243,29,29,0.00,MEMBER
1034327,1033458,15,15,0.00,MEMBER
1034328,1033520,33,33,0.00,MEMBER
1034329,1033884,59,59,0.00,MEMBER


In [ ]:
# ORDER BY + LIMIT -- find the longest trips
import time
start_time = time.time()
df = pd.read_sql("""
    SELECT trip_id,
           start_station_id,
           end_station_id,
           ROUND(duration_sec / 3600.0, 2) AS duration_hours,
           rider_type
    FROM   trips
    ORDER  BY duration_sec DESC
    LIMIT  100
""", conn)
print(f"Query executed in {time.time() - start_time:.5f} seconds")
df

Query executed in 1.63171 seconds


,trip_id,start_station_id,end_station_id,duration_hours,rider_type
0,845850,51,24,358.52,CASUAL
1,154039,32,24,192.97,MEMBER
2,875915,21,57,167.63,MEMBER
3,986838,59,52,141.72,MEMBER
4,164884,21,24,134.15,MEMBER
...,...,...,...,...,...
1034326,945313,23,22,0.00,MEMBER
1034327,967263,47,47,0.00,CASUAL
1034328,977610,25,24,0.00,MEMBER
1034329,991579,14,60,0.00,MEMBER


## 6  JOIN -- Combining Tables

Right now `trips` stores only station IDs. To see the **names** we need to **join** with the `stations` table.  
Each trip has *two* stations, so we join `stations` **twice** -- aliased `s1` (start) and `s2` (end).

In [75]:
pd.read_sql("""
    SELECT t.trip_id,
           s1.station_name  AS start_station,
           s2.station_name  AS end_station,
           t.duration_sec,
           t.rider_type,
           t.start_time
    FROM   trips    t
    JOIN   stations s1 ON t.start_station_id = s1.station_id
    JOIN   stations s2 ON t.end_station_id   = s2.station_id
    LIMIT  10
""", conn)

,trip_id,start_station,end_station,duration_sec,rider_type,start_time
0,1,Coltart Ave & Forbes Ave,Boulevard of the Allies & Parkview Ave,299,MEMBER,2024-04-30 23:53:00
1,2,Fifth Ave & S Bouquet St,Zulema St & Coltart Ave,399,MEMBER,2024-04-30 23:49:00
2,3,N Dithridge St & Centre Ave,Boulevard of the Allies & Parkview Ave,409,MEMBER,2024-04-30 23:46:00
3,4,Zulema St & Coltart Ave,Zulema St & Coltart Ave,348,MEMBER,2024-04-30 23:43:00
4,5,S Bouquet Ave & Sennott St,Boulevard of the Allies & Parkview Ave,248,MEMBER,2024-04-30 23:42:00
5,6,O'Hara St and University Place,N Dithridge St & Centre Ave,203,MEMBER,2024-04-30 23:40:00
6,7,S Bouquet Ave & Sennott St,Atwood St & Bates St,113,MEMBER,2024-04-30 23:23:00
7,8,S Bouquet Ave & Sennott St,Ellsworth Ave & N Neville St,639,MEMBER,2024-04-30 23:23:00
8,9,10th St & Penn Ave,Burns White Center at 3 Crossings,1051,CASUAL,2024-04-30 23:22:00
9,10,10th St & Penn Ave,Burns White Center at 3 Crossings,1102,CASUAL,2024-04-30 23:21:00


In [81]:
# Longest trips with human-readable station names
pd.read_sql("""
    SELECT s1.station_name                    AS start_station,
           s2.station_name                    AS end_station,
           ROUND(t.duration_sec / 3600.0, 1)  AS duration_hours,
           t.rider_type
    FROM   trips    t
    JOIN   stations s1 ON t.start_station_id = s1.station_id
    JOIN   stations s2 ON t.end_station_id   = s2.station_id
    ORDER  BY t.duration_sec DESC
    LIMIT  10
""", conn)

,start_station,end_station,duration_hours,rider_type
0,Coltart Ave & Forbes Ave,Penn Ave & 33rd St,358.5,CASUAL
1,Ross St & Fourth Ave,Penn Ave & 33rd St,193.0,MEMBER
2,Liberty Ave & Stanwix St,52nd St & Butler St,167.6,MEMBER
3,Forbes Ave at TCS Hall (CMU Campus),Bedford Ave & Memory Ln,141.7,MEMBER
4,Liberty Ave & Stanwix St,Penn Ave & 33rd St,134.1,MEMBER
5,S 22nd St & Sidney St,Forbes Ave & Market Square,122.9,MEMBER
6,Liberty Ave & Gross St,Penn Ave & 33rd St,94.1,MEMBER
7,Liberty Ave & Stanwix St,N Dithridge St & Centre Ave,91.4,MEMBER
8,7th St & Penn Ave,Penn Ave & 33rd St,82.6,MEMBER
9,N Dithridge St & Centre Ave,Penn Ave & 33rd St,70.1,MEMBER


## 7  Aggregations -- GROUP BY

`GROUP BY` collapses many rows into summary rows.  
You choose what to **group on** and what to **aggregate** (COUNT, SUM, AVG, MIN, MAX).

In [83]:
# Compare MEMBER vs CASUAL riders
pd.read_sql("""
    SELECT rider_type,
           COUNT(*)                              AS num_trips,
           ROUND(AVG(duration_sec) / 60.0, 1)   AS avg_duration_min,
           ROUND(MAX(duration_sec) / 3600.0, 1)  AS max_duration_hours
    FROM   trips
    GROUP  BY rider_type
""", conn)

,rider_type,num_trips,avg_duration_min,max_duration_hours
0,CASUAL,80066,42.9,358.5
1,MEMBER,954265,10.9,193.0


In [84]:
# Top 10 busiest departure stations
pd.read_sql("""
    SELECT s.station_name,
           COUNT(*)  AS departures
    FROM   trips    t
    JOIN   stations s ON t.start_station_id = s.station_id
    GROUP  BY t.start_station_id
    ORDER  BY departures DESC
    LIMIT  10
""", conn)

,station_name,departures
0,S Bouquet Ave & Sennott St,101120
1,Atwood St & Bates St,63458
2,Boulevard of the Allies & Parkview Ave,62332
3,N Dithridge St & Centre Ave,59581
4,O'Hara St and University Place,57979
5,Zulema St & Coltart Ave,47329
6,Coltart Ave & Forbes Ave,46912
7,Fifth Ave & S Bouquet St,46718
8,Forbes Ave & Schenley Dr,42885
9,Allequippa St & Darragh St,32333


In [85]:
# HAVING filters groups AFTER aggregation (WHERE filters individual rows BEFORE)
# Stations with more than 30,000 departures
pd.read_sql("""
    SELECT s.station_name,
           COUNT(*) AS departures
    FROM   trips    t
    JOIN   stations s ON t.start_station_id = s.station_id
    GROUP  BY t.start_station_id
    HAVING departures > 30000
    ORDER  BY departures DESC
""", conn)

,station_name,departures
0,S Bouquet Ave & Sennott St,101120
1,Atwood St & Bates St,63458
2,Boulevard of the Allies & Parkview Ave,62332
3,N Dithridge St & Centre Ave,59581
4,O'Hara St and University Place,57979
5,Zulema St & Coltart Ave,47329
6,Coltart Ave & Forbes Ave,46912
7,Fifth Ave & S Bouquet St,46718
8,Forbes Ave & Schenley Dr,42885
9,Allequippa St & Darragh St,32333


In [86]:
# Which (start -> end) pairs are most popular?
pd.read_sql("""
    SELECT s1.station_name  AS start_station,
           s2.station_name  AS end_station,
           COUNT(*)         AS num_trips
    FROM   trips    t
    JOIN   stations s1 ON t.start_station_id = s1.station_id
    JOIN   stations s2 ON t.end_station_id   = s2.station_id
    GROUP  BY t.start_station_id, t.end_station_id
    ORDER  BY num_trips DESC
    LIMIT  10
""", conn)

,start_station,end_station,num_trips
0,Boulevard of the Allies & Parkview Ave,S Bouquet Ave & Sennott St,21719
1,Atwood St & Bates St,S Bouquet Ave & Sennott St,19084
2,S Bouquet Ave & Sennott St,Atwood St & Bates St,18964
3,S Bouquet Ave & Sennott St,Boulevard of the Allies & Parkview Ave,18365
4,N Dithridge St & Centre Ave,O'Hara St and University Place,17433
5,O'Hara St and University Place,N Dithridge St & Centre Ave,15811
6,S Bouquet Ave & Sennott St,Zulema St & Coltart Ave,12936
7,Zulema St & Coltart Ave,S Bouquet Ave & Sennott St,11880
8,Coltart Ave & Forbes Ave,S Bouquet Ave & Sennott St,8873
9,S Bouquet Ave & Sennott St,S Bouquet Ave & Sennott St,8661


## 8  Time-Based Queries

SQLite stores our timestamps as plain text in ISO format (`"2024-04-30 23:53:00"`).  
The built-in `strftime()` function extracts parts of the date for grouping and filtering.

In [87]:
# Trips by month
pd.read_sql("""
    SELECT strftime('%Y-%m', start_time) AS month,
           COUNT(*)                       AS num_trips
    FROM   trips
    GROUP  BY month
    ORDER  BY month
""", conn)

,month,num_trips
0,2024-01,14437
1,2024-02,25341
2,2024-03,27403
3,2024-04,36735
4,2024-05,31714
5,2024-06,35606
6,2024-07,35530
7,2024-08,45454
8,2024-09,77273
9,2024-10,71519


In [88]:
# Peak hour of day (0-23)
pd.read_sql("""
    SELECT CAST(strftime('%H', start_time) AS INTEGER) AS hour_of_day,
           COUNT(*)                                     AS num_trips
    FROM   trips
    GROUP  BY hour_of_day
    ORDER  BY hour_of_day
""", conn)

,hour_of_day,num_trips
0,0,17100
1,1,12634
2,2,7831
3,3,3837
4,4,2197
5,5,3811
6,6,10582
7,7,27038
8,8,36884
9,9,44144


In [89]:
# Seasonal comparison using CASE WHEN
pd.read_sql("""
    SELECT
        CASE
            WHEN CAST(strftime('%m', start_time) AS INTEGER) BETWEEN 6 AND 8  THEN 'Summer'
            WHEN CAST(strftime('%m', start_time) AS INTEGER) IN (12, 1, 2)    THEN 'Winter'
            ELSE 'Spring/Fall'
        END                                     AS season,
        COUNT(*)                                AS num_trips,
        ROUND(AVG(duration_sec) / 60.0, 1)     AS avg_duration_min
    FROM   trips
    GROUP  BY season
    ORDER  BY num_trips DESC
""", conn)

,season,num_trips,avg_duration_min
0,Spring/Fall,653036,12.4
1,Summer,262027,17.8
2,Winter,119268,9.2


## 9  How Indices Speed Things Up

### SQL vs. pandas for large-table lookups

When your data fits in RAM, pandas is perfectly fast — it does a vectorized scan over a numpy array.
The real advantage of SQL + indices shows up when:
- The dataset is **larger than available RAM** (pandas can't even load it)
- You need **repeated, selective queries** on a subset of rows
- You want **O(log n) lookups** instead of O(n) scans

Let's time the same query three ways:
1. **pandas** — in-memory vectorized scan  
2. **SQLite, no index** — full table scan from disk  
3. **SQLite, with index** — B-tree seek from disk

In [91]:
query = "SELECT COUNT(*) FROM trips WHERE rider_type = 'CASUAL'"

# ── 1. pandas in-memory scan ───────────────────────────────────────────────────
t0 = time.time()
n_pd = (trips_df['rider_type'] == 'CASUAL').sum()
pd_ms = (time.time() - t0) * 1000

# ── 2. SQLite full-table scan (drop the index to force it) ────────────────────
cur.execute("DROP INDEX IF EXISTS idx_trips_rider")
conn.commit()

t0 = time.time()
n_scan = cur.execute(query).fetchone()[0]
scan_ms = (time.time() - t0) * 1000

# ── 3. SQLite with index ───────────────────────────────────────────────────────
cur.execute("CREATE INDEX idx_trips_rider ON trips(rider_type)")
conn.commit()

t0 = time.time()
n_idx = cur.execute(query).fetchone()[0]
idx_ms = (time.time() - t0) * 1000

# ── Summary ────────────────────────────────────────────────────────────────────
import pandas as pd
results = pd.DataFrame({
    'method':  ['pandas (in-memory)',  'SQLite — no index (full scan)', 'SQLite — with index'],
    'time_ms': [round(pd_ms, 1),       round(scan_ms, 1),              round(idx_ms, 1)],
    'rows':    [n_pd,                   n_scan,                          n_idx],
})
print(results.to_string(index=False))
print(f"\nIndex vs. scan speedup:   {scan_ms/idx_ms:.1f}x")
print(f"pandas requires all {len(trips_df):,} rows in RAM  |  SQLite reads only the matched rows from disk")

                       method  time_ms  rows
           pandas (in-memory)     39.4 80066
SQLite — no index (full scan)     46.5 80066
          SQLite — with index      1.7 80066

Index vs. scan speedup:   27.0x
pandas requires all 1,034,331 rows in RAM  |  SQLite reads only the matched rows from disk


### Key takeaways

| | pandas | SQLite (no index) | SQLite (with index) |
|---|---|---|---|
| Data location | RAM | Disk | Disk |
| RAM required | All rows | Negligible | Negligible |
| Lookup strategy | Vectorized scan O(n) | Full table scan O(n) | B-tree seek O(log n) |
| Best for | Exploration, transforms | Small/medium data | Repeated selective queries |

pandas `.query()` and boolean indexing always scan every row.  
A SQL index lets the database engine **skip** the rows it doesn't need — the larger the table and the more selective the filter, the bigger the win.


In [30]:
# Date-range query -- uses idx_trips_time
query = """
    SELECT COUNT(*)
    FROM   trips
    WHERE  start_time >= '2024-07-01'
      AND  start_time <  '2024-08-01'
"""

print("Query plan:")
for row in cur.execute(f"EXPLAIN QUERY PLAN {query}"):
    print(" ", row)

n = cur.execute(query).fetchone()[0]
print(f"\nJuly 2024 trips: {n:,}")

Query plan:
  (3, 0, 0, 'SEARCH trips USING COVERING INDEX idx_trips_time (start_time>? AND start_time<?)')

July 2024 trips: 35,530


## 10  Exercises

Try these on your own:

1. **Most popular end station** -- Which station receives the most arrivals?

2. **Round trips** -- How many trips start and end at the *same* station?  
   Hint: `WHERE start_station_id = end_station_id`

3. **Weekend vs. weekday** -- `strftime('%w', start_time)` returns `'0'` for Sunday, `'6'` for Saturday.  
   Compare trip counts between weekdays and weekends.

4. **Station over time** -- Pick your favourite station and count its monthly departures.

5. **Subquery** -- Find all trips that lasted longer than *twice* the average trip duration.

In [ ]:
# Exercise 1 -- most popular end station


In [ ]:
# Exercise 2 -- round trips


In [ ]:
# Exercise 3 -- weekend vs. weekday


In [ ]:
# Exercise 4 -- station popularity over time


In [ ]:
# Exercise 5 -- trips longer than 2x average duration


In [ ]:
# Always close the connection when you are done
conn.close()
print("Connection closed.")